# Yahtzee testing - Compute probabilities
The purpose of this part is to facilitate the computation of probabilities of rolls of Yahtzee, given any current state of the dice, including any frozen dice.

## 1. Imports needed

In [1]:
import numpy as np
import pandas as pd
import itertools as it
from __future__ import division

## 2. The table

This will go as follows:

1. Need to populate a column with every 5-digit combination of integers between 1 and 6.
2. Each row will be assigned probability of ${6^{-5}}$
3. A string representation of a dice roll will be passed, and joined to all of the rows which correspond to the dice roll. 
4. The probabilities of the rows will be summed up to the joined level, with an adjustment of $6^{k}$ applied if any $k$ dice are frozen.

In [2]:
print(list(it.permutations('123456',5))[0])

('1', '2', '3', '4', '5')


In [3]:
keys = list(it.product('123456',repeat=5))

In [4]:
keystr = [k[0] + k[1] + k[2] + k[3] + k[4] for k in keys]

In [5]:
keystr[0]

'11111'

In [6]:
rollprob = [1 / (6**5) for k in keys]

In [7]:
probs = pd.DataFrame({'key':keystr, 'rollprob':rollprob})

In [8]:
probs.head(3)

,key,rollprob
0,11111,0.000129
1,11112,0.000129
2,11113,0.000129


In [9]:
np.sum(probs.rollprob, axis=0)

0.9999999999998496

In [10]:
6**5 * np.sum(probs.rollprob, axis=0)

7775.99999999883

Whoa, these sum to something close to 1, but not exactly 1. That's weird. I'm going to proceed and see if that will be good enough.

# 3 Needed functions

## 3.1 parseRoll
This function will take a five character string of digits and return the values as ints in a list.

In [11]:
def parseRoll(roll):
    list = []
    if len(roll) != 5:
        return "Error in parseRoll!"
    else:
        for c in roll:
            list.append(int(c))
        return list

Test this function:

In [12]:
parseRoll('12345')

[1, 2, 3, 4, 5]

## 3.2 valueCounts
This function will take a five character string of digits and return a list of values where the index corresponds to  the possible values of the dice and the values are the number of times each die appears.

In [17]:
def valueCounts(roll):
    values = [0,0,0,0,0,0]
    list = parseRoll(roll)
    for val in list:
        values[val-1] += 1
    return values

Test this function:

In [18]:
valueCounts('12345')

[1, 1, 1, 1, 1, 0]

In [19]:
valueCounts('11246')

[2, 1, 0, 1, 0, 1]

## 3.3 makeRollfromList
This function will accept an array and glue it into a string representation of a roll.

In [33]:
def makeRollfromList(rollList):
    if len(rollList) != 5:
        return "Error in makeRollfromList!"
    else:
        rollStr = "".join(str(x) for x in rollList)
        return rollStr

Test this function:

In [35]:
makeRollfromList(['1','2','3','4','5'])

'12345'

## 3.4 probRoll
This function will accept a five character string representing a roll and calculate the probabillty of rolling that particular combination.

In [68]:
def probRoll(roll, probs):
    probroll = 0.0
    if len(roll) != 5:
        return "Error in probRoll!"
    else:
        rolls = set(it.permutations(roll, r=5))
        for s in rolls:
            print s
            p = probs.rollprob[probs.key==makeRollfromList(s)]
            print p
            probroll = probroll + float(p)
            print probroll
        return probroll

In [69]:
p = it.permutations('22333', r=5)
pset = set(p)
for d in pset: 
    print d

('2', '3', '2', '3', '3')
('3', '2', '3', '3', '2')
('2', '3', '3', '3', '2')
('2', '3', '3', '2', '3')
('3', '3', '2', '3', '2')
('2', '2', '3', '3', '3')
('3', '3', '2', '2', '3')
('3', '3', '3', '2', '2')
('3', '2', '3', '2', '3')
('3', '2', '2', '3', '3')


In [70]:
probs[probs.key=='11111']

,key,rollprob
0,11111,0.000129


In [71]:
probRoll('11111',probs)

('1', '1', '1', '1', '1')
0    0.000129
Name: rollprob, dtype: float64
0.000128600823045


0.0001286008230452675

In [72]:
probRoll('22333', probs)

('2', '3', '2', '3', '3')
1778    0.000129
Name: rollprob, dtype: float64
0.000128600823045
('3', '2', '3', '3', '2')
2893    0.000129
Name: rollprob, dtype: float64
0.000257201646091
('2', '3', '3', '3', '2')
1813    0.000129
Name: rollprob, dtype: float64
0.000385802469136
('2', '3', '3', '2', '3')
1808    0.000129
Name: rollprob, dtype: float64
0.000514403292181
('3', '3', '2', '3', '2')
3073    0.000129
Name: rollprob, dtype: float64
0.000643004115226
('2', '2', '3', '3', '3')
1598    0.000129
Name: rollprob, dtype: float64
0.000771604938272
('3', '3', '2', '2', '3')
3068    0.000129
Name: rollprob, dtype: float64
0.000900205761317
('3', '3', '3', '2', '2')
3103    0.000129
Name: rollprob, dtype: float64
0.00102880658436
('3', '2', '3', '2', '3')
2888    0.000129
Name: rollprob, dtype: float64
0.00115740740741
('3', '2', '2', '3', '3')
2858    0.000129
Name: rollprob, dtype: float64
0.00128600823045


0.0012860082304526753

# Full House testing

In [80]:
set(it.permutations('22333', r=5))

{('2', '2', '3', '3', '3'),
 ('2', '3', '2', '3', '3'),
 ('2', '3', '3', '2', '3'),
 ('2', '3', '3', '3', '2'),
 ('3', '2', '2', '3', '3'),
 ('3', '2', '3', '2', '3'),
 ('3', '2', '3', '3', '2'),
 ('3', '3', '2', '2', '3'),
 ('3', '3', '2', '3', '2'),
 ('3', '3', '3', '2', '2')}

In [82]:
set(it.permutations('233', r=3))

{('2', '3', '3'), ('3', '2', '3'), ('3', '3', '2')}